In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
from embedding import CustomEmbedding
from transformers import EncoderDecoderTransformer
from utils import padding_collate_fn

from generate_data import RandomIntegerDataset

In [27]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 16

embeddings = CustomEmbedding(vocab_size = vocab_size + 3, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[-1.0505, -0.1307, -0.5937, -2.0197, -0.4047, -0.8057, -2.2808, -0.7107,
          0.1728, -0.5888, -1.6933,  0.1606, -0.4092, -1.7453,  0.7205,  0.0121],
        [-0.2410, -0.6467, -0.5804, -0.4805,  0.8344, -1.0072,  0.9215, -1.0519,
          0.3204,  2.7408, -0.0303, -0.6972, -0.3721, -0.1038,  0.4547,  0.5299]],
       grad_fn=<EmbeddingBackward0>)


In [28]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 2
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(10, MAX_CONTEXT_WINDOW)

rand_ds = RandomIntegerDataset(2, 10, 10, [i for i in range(10)])
dataloader = DataLoader(rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [29]:
input, label = next(iter(dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 2,  5,  5,  3,  3,  6],
        [ 1,  7,  4,  8,  3, 10]])
tensor([[11,  2,  3,  3,  5,  5,  6],
        [11,  1,  3,  4,  7,  8, 10]])
tensor([[ 2,  3,  3,  5,  5,  6, 12],
        [ 1,  3,  4,  7,  8, 12, 10]])


In [30]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 1, 
                    num_decoder_layers = 1, 
                    dim_feedforward = 32, 
                    max_context_window = MAX_CONTEXT_WINDOW)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [31]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer):
    
    model.train()

    epoch_loss = 0.0

    for i, ((source, target), label) in enumerate(dataloader):
        print(f'Processing Batch {i+1}...')

        # FORWARD
        pred = model(source, target)
        print(pred)
        print(label)
        batch_loss = loss_fn(pred, label)

        # LOG
        epoch_loss += batch_loss.item()

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / len(dataloader.dataset)

    return average_epoch_loss

def run_test_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    
    model.eval()

In [32]:
EPOCHS = 1

training_losses = list()

for i in range(EPOCHS):

    training_loss = run_train_epoch(dataloader, model, loss_fn, optim)
    training_losses.append(training_loss)

Processing Batch 1...
tensor([[[0.0465, 0.0343, 0.1215, 0.0955, 0.0466, 0.0468, 0.0221, 0.1260,
          0.1409, 0.0919, 0.1460, 0.0244, 0.0574],
         [0.0270, 0.0356, 0.1417, 0.0631, 0.0948, 0.0537, 0.0227, 0.0556,
          0.1660, 0.0659, 0.1926, 0.0256, 0.0558],
         [0.0442, 0.0611, 0.0962, 0.1208, 0.0506, 0.1049, 0.0316, 0.0657,
          0.0850, 0.0768, 0.0958, 0.1349, 0.0323],
         [0.0814, 0.0390, 0.0936, 0.1175, 0.0383, 0.0440, 0.0212, 0.1415,
          0.0906, 0.1146, 0.0975, 0.0549, 0.0659],
         [0.0298, 0.0500, 0.0857, 0.1008, 0.0679, 0.0512, 0.0264, 0.0687,
          0.1582, 0.0515, 0.1981, 0.0531, 0.0587],
         [0.0376, 0.0695, 0.1233, 0.0771, 0.0877, 0.0245, 0.0791, 0.0207,
          0.1521, 0.0190, 0.1209, 0.1236, 0.0649],
         [0.0681, 0.0385, 0.1467, 0.0674, 0.0230, 0.0645, 0.0254, 0.0577,
          0.1957, 0.1202, 0.1123, 0.0343, 0.0463],
         [0.0688, 0.0371, 0.1469, 0.0701, 0.0214, 0.0730, 0.0258, 0.0641,
          0.1826, 0.1182, 0.1

RuntimeError: Expected target size [2, 13], got [2, 11]